In [1]:
# More detailed debug of Core_2 in test case 4
import os
from system import System
from analysis import Analysis

test_case_path = "DRTS_Project-Test-Cases-master/4-large-test-case"

# Initialize and analyze
system = System(
    os.path.join(test_case_path, "tasks.csv"),
    os.path.join(test_case_path, "architecture.csv"),
    os.path.join(test_case_path, "budgets.csv")
)

analyzer = Analysis(system)

# Analyze just Core_2 components
core_2 = system.cores["Core_2"]
core_2_components = [comp for comp in system.components.values() 
                     if comp.core_id == "Core_2"]

print("=== DETAILED CORE_2 ANALYSIS ===")
print(f"Core scheduler: {core_2.scheduler}")
print(f"Components: {[c.id for c in core_2_components]}")

# Manually compute BDR and check schedulability
for component in core_2_components:
    print(f"\n--- Component {component.id} ---")
    
    # Calculate utilization
    util = sum(task.wcet / (task.period * core_2.speed_factor) 
              for task in component.tasks)
    print(f"Utilization: {util:.6f}")
    
    # Get BDR parameters
    alpha, delta = analyzer.compute_optimal_bdr(component)
    component.bdr_alpha = alpha
    component.bdr_delta = delta
    
    print(f"BDR alpha: {alpha:.6f}")
    print(f"BDR delta: {delta:.6f}")
    
    # Check schedulability
    is_sched = analyzer._is_component_schedulable_with_bdr(component, alpha, delta)
    print(f"Component schedulable: {is_sched}")

# Now check core schedulability step by step
print("\n=== CORE SCHEDULABILITY CHECK ===")

# Step 1: Check all components schedulable
analyzer.analyze()  # Run full analysis

all_comp_sched = all(analyzer.analysis_results[comp.id]["schedulable"] 
                     for comp in core_2_components)
print(f"All components schedulable: {all_comp_sched}")

# Step 2: Calculate total alpha
total_alpha = sum(comp.bdr_alpha for comp in core_2_components)
print(f"Total alpha: {total_alpha:.6f}")
print(f"Total alpha > 1.0: {total_alpha > 1.0}")
print(f"Total alpha <= 1.001: {total_alpha <= 1.001}")

# Step 3: What would the check return?
if core_2.scheduler == "EDF":
    core_sched = all_comp_sched and total_alpha <= 1.001
    print(f"Core should be schedulable (EDF): {core_sched}")

# Step 4: What does the analyzer actually say?
core_2_result = analyzer.analysis_results.get("Core_2", {})
print(f"Analyzer says Core_2 schedulable: {core_2_result.get('schedulable', 'NOT FOUND')}")

# Let's check if there's an issue with the alpha values
print("\n=== ALPHA VALUES CHECK ===")
for comp in core_2_components:
    print(f"{comp.id} alpha from object: {comp.bdr_alpha:.6f}")
    print(f"{comp.id} alpha from results: {analyzer.analysis_results[comp.id]['bdr_alpha']:.6f}")

loaded dataframes: 
  core_id  speed_factor scheduler
0  Core_1          0.54       EDF
1  Core_2          0.70       EDF
2  Core_3          0.74        RM
         component_id scheduler  budget  period core_id  priority
0       Camera_Sensor        RM       4      11  Core_1       NaN
1     Image_Processor       EDF       2       7  Core_1       NaN
2    Bitmap_Processor        RM       1       7  Core_1       NaN
3        Lidar_Sensor        RM       1       3  Core_2       NaN
4        Control_Unit       EDF       4       6  Core_2       NaN
5          GPS_Sensor        RM       3      13  Core_3       1.0
6  Communication_Unit        RM       2       4  Core_3       0.0
   task_name  wcet  period        component_id  priority
0     Task_0     4     400       Camera_Sensor       3.0
1     Task_1     8     600       Camera_Sensor       4.0
2     Task_2     3      75       Camera_Sensor       0.0
3     Task_3     2     100       Camera_Sensor       1.0
4     Task_4     2     150     